In [2]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 398.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [5]:
import os
import time
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import glob

In [6]:
# Configuração do Streamlit
st.set_page_config(layout="wide", page_title="Monitoramento de Logs em Tempo Real")

# Função para carregar os logs continuamente
def load_logs_realtime(log_dir, last_loaded):
    log_files = glob.glob(log_dir + "/*/logs.txt")
    new_logs = []
    for file in log_files:
        category = file.split("/")[-2]
        with open(file, "r") as f:
            for line in f.readlines():
                try:
                    timestamp, _, log_level, message = line.strip().split(" - ", maxsplit=3)
                    log_time = datetime.strptime(timestamp.strip("[]"), "%Y-%m-%d %H:%M:%S")
                    if log_time > last_loaded:
                        new_logs.append({"Timestamp": log_time, "Category": category, "Level": log_level, "Message": message})
                except ValueError:
                    continue
    return pd.DataFrame(new_logs)

# Configurar o diretório de logs
log_dir = "/content/drive/MyDrive/Projeto Monitoramento/logs_demo"
last_loaded_time = datetime.now() - pd.Timedelta(days=30)  # Carregar logs dos últimos 30 dias
all_logs = pd.DataFrame()

st.title("Monitoramento de Logs - Tempo Real")
st.sidebar.header("Configurações de Tempo Real")
refresh_rate = st.sidebar.slider("Taxa de Atualização (segundos)", 1, 10, 5)

# Loop principal para atualização em tempo real
while True:
    # Carregar novos logs
    new_logs = load_logs_realtime(log_dir, last_loaded_time)
    if not new_logs.empty:
        all_logs = pd.concat([all_logs, new_logs], ignore_index=True)
        last_loaded_time = max(all_logs["Timestamp"])

2024-11-20 12:03:51.171 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-20 12:03:51.184 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-20 12:03:51.186 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-20 12:03:51.189 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-20 12:03:51.192 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-20 12:03:51.195 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-20 12:03:51.197 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-20 12:03:51.200 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

KeyboardInterrupt: 

In [ ]:




    # Processar dados
    if not all_logs.empty:
        all_logs["Date"] = all_logs["Timestamp"].dt.date
        all_logs["Hour"] = all_logs["Timestamp"].dt.hour

        # Exibir estatísticas principais
        st.subheader("Estatísticas Gerais")
        st.write(f"Total de Logs: {len(all_logs)}")
        st.write(f"Logs Críticos (ERROR): {len(all_logs[all_logs['Level'] == 'ERROR'])}")

        # Gráfico de barras: Logs por categoria
        st.subheader("Logs por Categoria")
        category_counts = all_logs["Category"].value_counts()
        fig1, ax1 = plt.subplots(figsize=(10, 5))
        category_counts.plot(kind="bar", ax=ax1)
        ax1.set_title("Logs por Categoria")
        ax1.set_xlabel("Categoria")
        ax1.set_ylabel("Número de Logs")
        st.pyplot(fig1)

        # Heatmap: Ocorrências por hora
        st.subheader("Ocorrências de Logs por Hora")
        hourly_logs = all_logs.groupby(["Hour", "Category"]).size().unstack(fill_value=0)
        fig2, ax2 = plt.subplots(figsize=(10, 5))
        sns.heatmap(hourly_logs, cmap="coolwarm", ax=ax2)
        ax2.set_title("Heatmap de Ocorrências por Hora")
        st.pyplot(fig2)

        # Destaques de eventos críticos
        st.subheader("Eventos Críticos")
        critical_events = all_logs[all_logs["Level"] == "ERROR"].sort_values("Timestamp", ascending=False).head(10)
        st.write(critical_events)

    # Pausar antes de atualizar
    time.sleep(refresh_rate)
